### Config stuff

In [1]:
import os

import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

Dynamically set JAVA_HOME: /Users/user/Library/Java/JavaVirtualMachines/temurin-21.0.2/Contents/Home


In [2]:
spark = cc.startLocalCluster("FACT_RIDE")
spark.getActiveSession()

:: loading settings :: url = jar:file:/Users/user/Desktop/spark_and_hadop/spark-3.5.4-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/user/.ivy2/cache
The jars for the packages stored in: /Users/user/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b0d8ce68-69ab-4e5d-8b8b-6cb7aeeeb488;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.

# Fact transformations
This notebooks creates the sales fact table from scratch based on the operational source table "sales"
When creating a fact table always follow the listed steps in order.


#### 1 READ NECESSARY SOURCE TABLE(S) AND PERFORM TRANSFORMATIONS
**When reading from the source table make sure you include all data necessary:**
- to calculate the measure values
- the source table keys that you have to use to lookup the correct surrogate keys in the dimension tables.

**If more than one table is needed to gather the necesary information you can opt for one of two strategies:**
- Use a select query when reading from the jdbc source with the spark.read operation. Avoid complex queries because the operational database needs a lot of resources to run those queries.
- Perform a spark.read operation for each table separately and join the tables within Spark. The joins will take place on the cluster instead of the database. You limit the database recources used, but there can be a significant overhead of unnecessary data tranferred to the cluster.


In this case we just rename Amount and create a default count_mv column.
The transformations are minimal. In reality, transformations can be far more complex. If so, it can be advisable to work out the transforms in more then one step.*



In [10]:
cc.set_connectionProfile("default")
ride_src_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable","rides").option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()

subscriptions_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable","subscriptions").option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "subscriptionid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1000) \
    .load()

# weather_df = spark.read.json(r'C:\Users\kkiva\data4_project_group5\examples\weather\*.json')


In [4]:
subscriptions_df.show(20)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/user/Desktop/data4_project_group5/myenv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/user/Desktop/data4_project_group5/myenv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 


#### 2 MAKE DIMENSION TABLES AVAILABLE AS VIEWS

In [11]:
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")
dim_vehicle = spark.read.format("delta").load("spark-warehouse/dimvehicle")
dim_user = spark.read.format("delta").load("spark-warehouse/dimuser")
dim_weather = spark.read.format("delta").load("spark-warehouse/dimuser")
dim_lock = spark.read.format("delta").load("spark-warehouse/dimlock")

dim_date.createOrReplaceTempView("dimDate")
dim_user.createOrReplaceTempView("dimUser")
dim_vehicle.createOrReplaceTempView("dimVehicle")
dim_weather.createOrReplaceTempView("dimWeather")
dim_lock.createOrReplaceTempView("dimLock")

In [37]:
# Function to safely show table information
def show_table_info(table_name, df):
    print(f"\n=== {table_name} ===")
    print(f"Row count: {df.count()}")
    print("Schema:")
    df.printSchema()
    
    # Show just a few rows to avoid memory issues
    print("Sample data (5 rows):")
    df.limit(5).show(truncate=False)
    
    # Give Spark a moment between operations
    import time
    time.sleep(1)


show_table_info("user sub", subscriptions_df)


=== user sub ===
Row count: 78248
Schema:
root
 |-- subscriptionid: integer (nullable = true)
 |-- validfrom: date (nullable = true)
 |-- subscriptiontypeid: integer (nullable = true)
 |-- userid: integer (nullable = true)

Sample data (5 rows):
+--------------+----------+------------------+------+
|subscriptionid|validfrom |subscriptiontypeid|userid|
+--------------+----------+------------------+------+
|1             |2019-08-02|3                 |1     |
|2             |2019-11-12|1                 |1     |
|3             |2020-12-14|1                 |1     |
|4             |2021-10-05|2                 |2     |
|5             |2022-09-17|3                 |3     |
+--------------+----------+------------------+------+




#### 3 Build the fact table

Within the creation of a fact table always perform these two tasks:
1.   Include the measures of the fact
2. Use the dimension tables to look up the surrogate keys that correspond with the natural key value. In case of SCD2 dimension use the scd_start en scd_end to find the correct version of the data in the dimension


In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import math

def haversine_km(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2):  # Handle NULL values safely
        return None  
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Register the UDF in Spark SQL
haversine_udf = udf(haversine_km, DoubleType())
spark.udf.register("haversine_km", haversine_udf)


25/03/29 16:49:52 WARN SimpleFunctionRegistry: The function haversine_km replaced a previously registered function.


In [34]:
ride_src_df.createOrReplaceTempView("rides_source")
subscriptions_df.createOrReplaceTempView("subscriptions_source")

ridesFactFromSource = spark.sql("""
    SELECT 
        src.rideid AS ride_id, 
        du.userSK AS user_sk, 
        src.startlockid AS start_lock_id, 
        src.endlockid AS end_lock_id, 
        dd.date_sk AS date_sk, 
        dv.vehicleid AS vehicle_id, 
        (src.endtime - src.starttime) AS ride_duration, 
        CASE 
            WHEN dl_start.gpscoord IS NULL OR dl_end.gpscoord IS NULL THEN NULL 
            ELSE haversine_km(
                CAST(SPLIT(REPLACE(dl_start.gpscoord, '(', ''), ',')[0] AS DOUBLE),
                CAST(SPLIT(REPLACE(REPLACE(dl_start.gpscoord, ')', ''), '(', ''), ',')[1] AS DOUBLE),
                CAST(SPLIT(REPLACE(dl_end.gpscoord, '(', ''), ',')[0] AS DOUBLE),
                CAST(SPLIT(REPLACE(REPLACE(dl_end.gpscoord, ')', ''), '(', ''), ',')[1] AS DOUBLE)
            )
        END AS distance_km,
        md5(concat(src.rideid, du.userSK, src.startlockid, src.endlockid, dd.date_sk, dv.vehicleid)) AS md5 
    FROM rides_source AS src 
    LEFT OUTER JOIN subscriptions_source AS sub ON src.subscriptionid = sub.subscriptionid 
    LEFT OUTER JOIN dimUser AS du ON sub.userid = du.userSK 
    LEFT OUTER JOIN dimLock AS dl_start ON src.startlockid = dl_start.lockid
    LEFT OUTER JOIN dimLock AS dl_end ON src.endlockid = dl_end.lockid
    LEFT OUTER JOIN dimDate AS dd ON DATE(src.starttime) = dd.date 
    LEFT OUTER JOIN dimVehicle AS dv ON src.vehicleid = dv.vehicleid
""")


In [ ]:
ride_src_df.createOrReplaceTempView("rides_source")
subscriptions_df.createOrReplaceTempView("subscriptions_source")

In [41]:
xd = spark.sql("""
SELECT src.subscriptionid 
FROM rides_source AS src
LEFT JOIN subscriptions_source AS sub ON sub.subscriptionid = src.subscriptionid
WHERE sub.subscriptionid IS NULL;

""")
xd.show()

+--------------+
|subscriptionid|
+--------------+
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
|          NULL|
+--------------+
only showing top 20 rows



In [35]:
ridesFactFromSource.printSchema()
ridesFactFromSource.show(20)

root
 |-- ride_id: integer (nullable = true)
 |-- user_sk: string (nullable = true)
 |-- start_lock_id: integer (nullable = true)
 |-- end_lock_id: integer (nullable = true)
 |-- date_sk: long (nullable = true)
 |-- vehicle_id: integer (nullable = true)
 |-- ride_duration: interval day to second (nullable = true)
 |-- distance_km: double (nullable = true)
 |-- md5: string (nullable = true)



+-------+-------+-------------+-----------+-------+----------+--------------------+------------------+----+
|ride_id|user_sk|start_lock_id|end_lock_id|date_sk|vehicle_id|       ride_duration|       distance_km| md5|
+-------+-------+-------------+-----------+-------+----------+--------------------+------------------+----+
|      3|   NULL|         2046|       1951|   2455|      NULL|INTERVAL '-1095 0...|1.4949081239108692|NULL|
|     11|   NULL|          985|       2148|   2455|      NULL|INTERVAL '-1095 0...| 2.067612187914813|NULL|
|     13|   NULL|         5619|       2717|   2455|      NULL|INTERVAL '-1095 0...| 5.381957169382341|NULL|
|     17|   NULL|         2046|       1951|   3916|      NULL|INTERVAL '0 00:02...|1.4949081239108692|NULL|
|      1|   NULL|         4849|       3188|   2455|      NULL|INTERVAL '-1095 0...| 3.491018931475966|NULL|
|      4|   NULL|         1821|       2186|   2455|      NULL|INTERVAL '-1095 0...|2.3114308809290756|NULL|
|      5|   NULL|         63

## Initial load
The first time loading the fact table perform a FULL load. All data is written to the Delta Table.
After initial load the code line has to be disabled

In [6]:

ridesFactFromSource.write.format("delta").mode("overwrite").saveAsTable("factRides")



## Incremental load
When previous runs where performend you can opt for a 'faster' incremental run that only writes away changes. UPDATES and INSERTS are performed in one run.
In our solution we use an md5 based on all fields in the source table to detect changes. This is not the most efficient way to detect changes. A better way is to use a timestamp field in the source table and use that to detect changes. This is not implemented in this example.

In [7]:
dt_factRides = DeltaTable.forPath(spark,".\spark-warehouse\\factrides")
dt_factRides.toDF().createOrReplaceTempView("factRides_current")
#Merge to perform updates (TODO: Implement md5 strategy)

result = spark.sql("MERGE INTO factRides_current AS target \
      using factRides_new AS source ON target.rideID = source.rideID \
      WHEN MATCHED and source.MD5<>target.MD5 THEN UPDATE SET * \
      WHEN NOT MATCHED THEN INSERT *")

result.show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                0|               0|               0|                0|
+-----------------+----------------+----------------+-----------------+


In [8]:
# IMPORTANT: ALWAYS TEST THE CREATED CODE.
# In this example I changed order 498 in the operational database and checked the change after the run.
# spark.sql("select * from factsales f join dimsalesrep ds on f.salesrepSK = ds.salesrepSK where OrderID = 192  ").show()
spark.sql("select count(*) from factrides").show()
spark.sql("select * from factrides where rideId=1").show()



+--------+
|count(1)|
+--------+
|     999|
+--------+

+-------+------+--------------------+--------+----------+--------------------+
|OrderID|dateSK|          salesrepSK|count_mv|revenue_mv|                 md5|
+-------+------+--------------------+--------+----------+--------------------+
|      1|   650|b65df3d9-20dc-42d...|       1| 851804379|a237b06f2932af7dd...|
+-------+------+--------------------+--------+----------+--------------------+


### Checking the history of your delta fact table

In [9]:
# The history information is derived from the delta table log files. They contain a lot of information of all the actions performed on the table. In this case it tells us something about de merge operations. You can find statistics about the update and insert counts in the document.

fact.history().show(10,False)

NameError: name 'fact' is not defined

In [13]:
spark.stop()